In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_bid, close_trading
from store.operations import StoringPool
from utils import get_algod_client, wait_for_confirmation, get_balances, get_last_block_timestamp
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
def optin_to_asset(asset_id, sender: Account):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    wait_for_confirmation(client, signed_txn.get_txid())

In [4]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 742N2DLQ6NUUQXNUTNIJHYXYGL6BWG7C2XSQDHLFJN2NFYDVAYYUAQ36NQ
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [5]:
def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        decimals=decimals,
        asset_name=asset_name,
        unit_name=unit_name,
        default_frozen=False,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")
print(f"The token id is: {asset_id}")

Waiting for confirmation...
Transaction HM6I6UMSNLFELLS54VRIF2AP2MCZFPMT67EC66VVVS5Q5BCUMVGA confirmed in round 5381.
The token id is: 265


In [6]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [7]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)
storeApp.create_app()
storeAppID = storeApp.app_id

appID = create_trading_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    price=price,
    store_app_address=get_application_address(storeAppID)
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction VPDESAE3CQPI4HQB7NZFKNIDTRX2X5Z4RW7Q5RGPJCWXO72SZDIA confirmed in round 5383.
Store App ID: 266
Store App address: ZYE6MAYVU4OODNGXFUO5I7K5OTW2KT7JY6HSU7FB6BVO4D2NCRYW4POM4A
Waiting for confirmation...
Transaction OSVHSXXXIDMTEBYJD4UPH6ZEI42VDNYXU4UZTTPUZG3I2LHPCO3A confirmed in round 5385.
Waiting for confirmation...
Transaction JY5MDTO5O3PMKDT6KKICWUGYEE7DQ7WSEWNX3NW7YT4C7LFUK7OA confirmed in round 5387.
App ID: 268
App Address: 7Z2XRQGMXUY2MRKMSFURRGLC4KCAOIFCB7SW47BXJBCSBEMPQJNLPD3WHA


Alice is setting up and funding token auction...

In [8]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=1
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction NZCEG6C5N5K3G2CTKH26CCD5Y3YG42WGRX7KAWBVIVZZ5I5ALTDA confirmed in round 5389.
Alice's algo balance:  20001321090118  algos


In [9]:
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))

print(f"Bidder Address: {bidder.get_address()}")

Bidder Address: GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI


In [10]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 202000, 265: 1}


In [11]:
bidAmount = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

Carla wants to bid on token, her algo balance:  10001299013246  algos
Carla is placing bid for:  1000000  algos


In [12]:
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

Waiting for confirmation...
Transaction YRAGJXMWDL2WY3LNABYASSCTWNAI2QVDEOD3NFF73I4ICVCJK3YA confirmed in round 5392.
Carla is opting into token with id: 265


In [13]:
optin_to_asset(asset_id, bidder)

Waiting for confirmation...
Transaction 5SN54HTUX4GVPC4M2UFRI7ZORJMBK7XNFPXMUCRB6ZEUNI4VB4HQ confirmed in round 5394.


Alice is closing out the auction....

In [14]:
sleep(5)

close_trading(client, appID, seller)

b'token_id' 265
b'seller' Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM
b'accounts' ['Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM']
b'bidder' GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI
Waiting for confirmation...
Transaction 7WP7EYEPAJ234GJHBGFVYTI4JWOBSPTDYCCKLVNFYH74V5ES6NIQ confirmed in round 5397.


In [16]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 129092}


In [17]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 1  for token ID:  265
Alice's balances after auction:  {0: 20001362159732, 6: 0, 17: 0, 31: 0, 42: 0, 53: 0, 68: 0, 80: 0, 92: 0, 104: 1, 107: 0, 118: 0, 129: 0, 140: 0, 155: 0, 166: 0, 177: 0, 188: 1, 189: 1, 190: 1, 191: 1, 192: 1, 193: 1, 195: 0, 206: 0, 217: 1, 220: 0, 231: 0, 247: 0, 265: 0}  Algos
Carla's balances after auction:  {0: 10001318012842, 6: 0, 17: 0, 31: 0, 42: 0, 53: 0, 68: 0, 80: 0, 92: 0, 107: 0, 118: 0, 129: 0, 140: 0, 155: 0, 166: 0, 177: 0, 195: 0, 206: 0, 220: 0, 231: 0, 247: 1, 265: 1}  Algos
